# Algoritmo Naïve Bayes

Presentamos una implementación del algoritmo de Bayes naïve (o ingenuo) para un problema particular que la clasificación de texto en lenguas a partir de la frecuencia de los conjuntos de caracteres que en ellos se presentan.

In [1]:
from nltk.corpus import brown, cess_esp
from nltk import ngrams
from elotl.corpus import load
from itertools import chain
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter, defaultdict
from re import sub
import numpy as np

### Preparación de los datos

En primer lugar, obtenemos los datos con los que vamos a trabajar:

In [2]:
#Datos a usar en inglés, español, otomí y náhua
eng = brown.sents()
esp = cess_esp.sents()
oto = [sent[1].split() for sent in load('tsunkua')]
nah = [sent[1].split() for sent in load('axolotl')]

Para hacer la clasificaicón usaremos conjuntos de caracteres (n-gramas); es decir, un texto se clasificará en una lengua según los patrones de caracteres que contenga.

In [3]:
def get_ngrams(word,n):
    """
    Función para obtener n-gramas.
    
    Arguments
    ---------
    word : str
        Palabra que se va a separar en patrones de caracteres o n-gramas.
    n : int
        Tamaño de n-grama
        
    Returns
    -------
        Lista de n-gramas de la cadena.
    """
    #Limpia el texto
    clean_word = sub('[^\w\s)]','',word.lower())        
    if len(clean_word) <= n and clean_word != '':
        #Si no se peuden obtener n-gramas
        ngram_list = [clean_word]
    else:
        #Obtiene n-gramas
        ngram_list = [''.join(ngram) for ngram in  ngrams(clean_word,n)]
    
    return ngram_list

def process(sent, n=2):
    """
    Función para procesar las oraciones de un texto.
    
    Arguments
    ---------
    sent : list[str]
        Lista de palabras que componen una oración del texto
    n : int
        Tamaño de n-grama
        
    Returns
    -------
        Lista de n-gramas de la oración
    """
    sent_ngrams = list(chain(*[get_ngrams(w,n) for w in sent]))
    
    return sent_ngrams

Podemos observar qué tipo de procesamiento es el que se hace:

In [4]:
input_text = 'y el ¿que del niño de aquí?.'.split()
print(process(input_text))

['y', 'el', 'qu', 'ue', 'de', 'el', 'ni', 'iñ', 'ño', 'de', 'aq', 'qu', 'uí']

In [4]:
class Dataset(object):
    """
    Clase para crear el dataset de las lenguas.
    """
    def __init__(self):
        """
        Values
        ------
        self.language
            Diccionario de lenguas y sus correspondientes textos.
        self.X
            Datos de entrada
        self.Y
            Clases de salida
        """
        self.languages = {'english': eng,'spanish':esp,'nahuatl':nah,'otomi':oto}
        self.X = []
        self.Y = []
        
    def get_dataset(self):
        """
        Función para crear el dataset (pares [x,y]) a partir de los textos.
        """
        for lang, sentences in self.languages.items():
            print(lang)
            for sent in sentences:
                #Procesa los textos
                x = process(sent)

                #Genera los inputs y las clases
                self.X.append(x)
                self.Y.append(lang)

In [5]:
#Creamos el dataset
dataset = Dataset()
dataset.get_dataset()

english
spanish
nahuatl
otomi


In [6]:
#Imprime longitud del dataset
print(len(dataset.X), len(dataset.Y))

84450 84450


Creamos el conjunto de entrenamiento y de evaluación:

In [7]:
x_train, x_test, y_train, y_test = train_test_split(dataset.X, dataset.Y, test_size=0.3)

## Modelo de Naïve Bayes

Ahora definimos el modelo de Naïve Bayes con el cual podremos realizar la clasificación que esperamos. Este modelo guarda las probabilidades de la forma: $p(x|y)$ y $p(y)$; estos pueden pensarse como los parámetros del modelo a partir de los cuáles se puede estimar la probabilidad:

$$p(y,x) = \prod_i p(x_i|y)p(y)$$

Y la clase se obtiene como $\hat{y} = \arg\max_y p(y,x)$.

In [8]:
class NaiveBayesClassifier(object):
    """
    Clase del modelo de bayes ingenuo.
    """
    def __init__(self, priors={}):
        """
        Values
        ------
        self.prec_prior 
            Valores de probabilidad prior si es que se dan.
        self.priors
            Priors del modelo
        self.categories
            Categorias o clases del modelo.
        self.conditional
            Probabilidades de la forma p(x|y) para los rasgos.
            
        Arguments
        ---------
        priors : dict
            Dictionario de probabilidades a priori para cada clase, 
            si no se presenta, las prior se calculan de los ejemplos.
        """
        self.prec_priors = priors
        self.priors = {}
        categories = []
        self.conditional = {}
    
    def count_cat(self, y):
        """
        Función para contar las clases.
        
        Arguments
        ---------
        y : list
            Lista de clases para cada uno de los ejemplos.
        """
        freqs = Counter(y)
        total_freq = sum(freqs.values())
        for lang, freq in freqs.items():
            self.priors[lang] = freq/total_freq
            
    def count_cond(self,x,y):
        """
        Función para contar las probabilidades condicionales p(x|y)
        
        Arguments
        ---------
        x, y : list
            Lista de ejemplos de entrenamiento asociados a sus clases
        """
        freq_cat = Counter(y)
        print(freq_cat)
        joint_elements = defaultdict(list)
        for category,example in zip(y,x):
            joint_elements[category].append(example)
        
        for category,examples in joint_elements.items():
            freqs = Counter(chain(*examples))
            total_freq = sum(freqs.values())
            self.conditional[category] = {w:freq/total_freq for w,freq in freqs.items()}
    
    def fit(self,x,y):
        """
        Función para entrenar el modelo.
        
        Arguments
        ---------
        x, y : list
            Lista de ejemplos de entrenamiento asociados a sus clases
        """
        if self.prec_priors == {}:
            self.count_cat(y)
        else:
            for i,category in enumerate(set(y)):
                self.priors[category] = self.prec_priors[i]
        
        self.categories = list(self.priors.keys())
        self.count_cond(x,y)
        
    def predict_proba(self,x):
        """
        Función para obtener probabilidades de clases.
        
        Arguments
        ---------
        x : list
            Lista de ejemplos para predicción.
            
        Returns
        -------
            Probabilidad de las clases.
        """
        prediction = np.zeros(len(self.priors))
        for i,category in enumerate(self.categories):
            p = 1
            prior = self.priors[category]
            for x_i in x:
                try:
                    cond = self.conditional[category][x_i]
                except:
                    cond = 1/prior
                p *= cond*prior
                
            prediction[i] = p
            
        return prediction
    
    def predict_logproba(self,x):
        """
        Función para obtener logaritmos de probabilidades de clases.
        
        Arguments
        ---------
        x : list
            Lista de ejemplos para predicción.
        
        Returns
        -------
            Logaritmo de la probabilidad de las clases.
        """
        prediction = np.zeros(len(self.priors))
        for i,category in enumerate(self.categories):
            p = 0
            prior = self.priors[category]
            for x_i in x:
                try:
                    cond = self.conditional[category][x_i]
                except:
                    cond = 1/prior
                
                p += np.log(cond*prior)
                
            prediction[i] = p
            
        return prediction
        
    def predict(self,x,log=True):
        """
        Función para predecir las clases de un ejemplo de evaluación.
        
        Arguments
        ---------
        x : list
            Lista de ejemplos para predicción.
        log : bool
            Indica si se obtienen las clases desde el logaritmo de la probabilidad
            
        Returns
        -------
            Clase de mayor probabilidad.
        """
        if log:
            probas = self.predict_logproba(x)
        else:
            probas = self.predict_proba(x)
        y_hat = np.argmax(probas)
        
        return self.categories[y_hat]

Construimos el modelo en base a unos prios uniformes y lo entrenamos con nuestros datos de entrenamiento que hemos obtenido anteriormente.

In [9]:
#Modelo
clf = NaiveBayesClassifier(priors=[0.25,0.25,0.25,0.25])
#Entrenamiento
clf.fit(x_train,y_train)

Counter({'english': 40168, 'nahuatl': 11254, 'spanish': 4218, 'otomi': 3475})


### Evaluación del modelo

Para evaluar el modelo, usamos el dataset de evaluación y predecimos las clases a las que pertenece. Calculamos métricas de evaluación para saber qué tan bien trabaja nuestro modelo.

In [10]:
y_pred = [clf.predict(x) for x in x_test]
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     english       0.89      0.95      0.92     17172
     nahuatl       0.97      0.88      0.92      4863
       otomi       0.43      0.52      0.47      1488
     spanish       0.63      0.35      0.45      1812

    accuracy                           0.86     25335
   macro avg       0.73      0.67      0.69     25335
weighted avg       0.86      0.86      0.86     25335



Podemos ver cómo trabaja en casos individuales de clasificación.

In [11]:
#Texto de entrada
input_text = 'el gobierno' #'ra detha' #'tinechmacasnequi'

#Imprime la clase
print(clf.predict(process(input_text.split()), log=True))

spanish


Finalmente, podemos explorar cuáles son los rasgos que más influyen para la decisión en cada una de las clases:

In [12]:
from operator import itemgetter
sorted(clf.conditional['spanish'].items(),key=itemgetter(1),reverse=True)[:10]

[('de', 0.02949939931099778),
 ('en', 0.024363707245661913),
 ('es', 0.022261295737892737),
 ('la', 0.01858157834018888),
 ('os', 0.017807843168455472),
 ('er', 0.016345901391529888),
 ('el', 0.01600975423465856),
 ('ar', 0.01519623833430133),
 ('ra', 0.014860091177430005),
 ('ue', 0.014846167922411667)]